<u><b>The Code below is to be ran in colab and so the shell command (!) keys won't work in a normal notebook:</b></u>  

In [ ]:
# -------------------------------------------------------------
# 1. GPU Setup
# -------------------------------------------------------------
!nvidia-smi
#or fallback:
# import subprocess
# subprocess.run(["nvidia-smi"])

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

# -------------------------------------------------------------
# 2. Pull Latest Repo
# -------------------------------------------------------------
!git clone "https://github.com/Zed-080/NeuralSpikeSorter-2009.git"

# or fallback 
# subprocess.run(["git", "clone", "https://github.com/Zed-080/NeuralSpikeSorter-2009.git"])

# -------------------------------------------------------------
# 3. Switch focus to repo and Install Dependencies
# -------------------------------------------------------------
%cd NeuralSpikeSorter-2009

# or alternatively (more lines)
# import os
# os.chdir('NeuralSpikeSorter-2009')
# print("Working directory set to:", os.getcwd())

!pip install -r requirements.txt

# or fallback:
# subprocess.run(["pip", "install", "-r", "requirements.txt"])

# or manually:
# !pip install numpy scipy scikit-learn tensorflow matplotlib


# -------------------------------------------------------------
# 4. Run Scripts
# -------------------------------------------------------------
!python main_train.py
!python main_infer.py

# or fallback:
# subprocess.run(["python", "main_train.py"])
# subprocess.run(["python", "main_infer.py"])

# -------------------------------------------------------------
# 5. Save Results to Google Drive (unique filename each run)
# -------------------------------------------------------------
from google.colab import drive
import shutil, datetime

drive.mount('/content/drive')

def save_results(dataset="D1-D6", threshold=0.95, refractory=60, folder="Colab Notebooks"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    threshold_str = str(threshold).replace('.', '_')
    zip_name = f"results_{dataset}_thr{threshold_str}_refr{refractory}_{timestamp}"
    output_path = f"/content/drive/MyDrive/{folder}/{zip_name}"
    shutil.make_archive(output_path, 'zip', 'outputs')
    print(f"Saved archive: {output_path}.zip")

# Example usage:
save_results(dataset="D2-D6", threshold=0.82, refractory=45)

# -------------------------------------------------------------
# 6. Commit Results to GitHub (outputs + metadata)
# -------------------------------------------------------------
import subprocess, datetime

def commit_results(threshold=0.95, refractory=60, script_run="main_train.py + main_infer.py"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name --format=csv,noheader")
    threshold_str = str(threshold).replace('.', '_')

    commit_message = (
        f"Run at {timestamp} | GPU: {gpu_info} | Scripts: {script_run} | "
        f"Threshold={threshold_str} | Refractory={refractory}"
    )

    # Add and commit outputs folder
    !git add outputs/*
    !git commit -m "{commit_message}"
    !git push origin main

# Example usage:
commit_results(threshold=0.82, refractory=45)


In [ ]:
import os, shutil, datetime
from google.colab import drive, files  # type: ignore

# 1) Ensure Google Drive is mounted
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

# 2) Define source folder (inside repo) and destination (outside repo in Drive)
src_folder = "NeuralSpikeSorter-2009/submission_datasets"
base_output = "/content/drive/MyDrive/ColabOutputs/SubmissionDatasets"
os.makedirs(base_output, exist_ok=True)

# 3) Create a timestamped archive name
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
zip_name = f"submission_datasets_{timestamp}"
zip_path = os.path.join(base_output, zip_name)

# 4) Create the archive directly in Google Drive
shutil.make_archive(zip_path, 'zip', src_folder)
print(f"✅ Saved archive to Drive: {zip_path}.zip")

# 5) Optional: also download to your local machine (uncomment to use)
files.download(f"{zip_path}.zip")


<u><b>Clean up runtime worksapce:</b></u>

In [ ]:
# Cleanup 
import os

# Get current working directory
cwd = os.getcwd()

# Define repo folder name
repo_folder = "NeuralSpikeSorter-2009"

# Full path to repo
repo_path = os.path.join("/content", repo_folder)

# Cleanup logic
if cwd.endswith(repo_folder):
    print(f"✅ Already inside '{repo_folder}' — no cleanup needed.")
elif os.path.exists(repo_path):
    shutil.rmtree(repo_path)
    print(f"🧹 Removed '{repo_folder}' from workspace.")
else:
    print(f"ℹ️ '{repo_folder}' not found — nothing to clean.")
